In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [4]:
df_jan = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_feb = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [5]:
df_jan.columns.__len__()

19

In [6]:
df_jan['duration'] = (df_jan['tpep_dropoff_datetime'] - df_jan['tpep_pickup_datetime']).dt.total_seconds() / 60
df_feb['duration'] = (df_feb['tpep_dropoff_datetime'] - df_feb['tpep_pickup_datetime']).dt.total_seconds() / 60

In [7]:
df_jan['duration'].describe()

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.002918e+04
Name: duration, dtype: float64

In [8]:
((df_jan['duration']>=1) & (df_jan['duration']<=60)).mean()

0.9812202822125979

In [9]:
df_jan = df_jan[(df_jan['duration']>=1) & (df_jan['duration']<=60)]
df_feb = df_feb[(df_feb['duration']>=1) & (df_feb['duration']<=60)]

In [11]:
cats = ['DOLocationID', 'PULocationID']
df_jan[cats] = df_jan[cats].astype(str)
train_dicts = df_jan[cats].to_dict(orient = 'records')

In [13]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
y_train = df_jan['duration']

In [23]:
X_train.shape

(3009173, 515)

In [16]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [22]:
y_pred = model.predict(X_train)
rmse = np.sqrt(mean_squared_error(y_train, y_pred))
print(rmse)

7.649261930819891


### Test

In [19]:
df_feb[cats] =  df_feb[cats].astype(str)
test_dicts = df_feb[cats].to_dict(orient = 'records')
X_test = dv.transform(test_dicts)
y_test  = df_feb['duration']
y_pred = model.predict(X_test)

In [21]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(rmse)

7.811817675774269
